In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%pip install -q -U 'google-generativeai>=0.8.3'

Note: you may need to restart the kernel to use updated packages.


In [3]:
import google.generativeai as genai
from IPython.display import Markdown, HTML, display

In [6]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [7]:
for model in genai.list_models():
    if "002" in model.name:
        print(model.name)

models/gemini-1.5-pro-002
models/gemini-1.5-flash-002


In [8]:
# Ask for information without search grounding.
model = genai.GenerativeModel("gemini-1.5-flash-002")

response = model.generate_content("When and where is Taylor Swift's next concert?")

Markdown(response.text)

I do not have access to real-time information, including live event schedules like concert dates.  To find the date and location of Taylor Swift's next concert, you should check these resources:

* **Taylor Swift's official website:** This is the most reliable source.
* **Ticketmaster or other ticket vendors:** Search for Taylor Swift tickets on these sites.
* **Social media:** Check Taylor Swift's official social media accounts for announcements.


Keep in mind that concert dates are constantly changing, so checking frequently is a good idea.


In [9]:
# And now re-run the same query with search grounding enabled.
model = genai.GenerativeModel(
    "gemini-1.5-flash-002",
    tools="google_search_retrieval")

response = model.generate_content("When and where is Taylor Swift's next concert?")
rc = response.candidates[0]

Markdown(rc.content.parts[0].text)

ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [ ]:
# Response metadata¶
# When search grounding is used, the model returns extra metadata that includes links to search suggestions, supporting documents and 
# information on how the supporting documents were used.Each "grounding chunk" represents information retrieved from Google Search that
# was used in the grounded generation request. Following the URI will take you to the source.

chunks = rc.grounding_metadata.grounding_chunks
for chunk in chunks:
    print(chunk)

In [ ]:
supports = rc.grounding_metadata.grounding_supports
for support in supports:
    print(support)

In [ ]:
import io

markdown_buffer = io.StringIO()

# Print the text with footnote markers.
markdown_buffer.write("Supported text:\n\n")
for support in supports:
    markdown_buffer.write(" * ")
    markdown_buffer.write(
        response.text[support.segment.start_index : support.segment.end_index]
    )

    for i in support.grounding_chunk_indices:
        chunk = chunks[i].web
        markdown_buffer.write(f"<sup>[{i+1}]</sup>")

    markdown_buffer.write("\n\n")


# And print the footnotes.
markdown_buffer.write("Citations:\n\n")
for i, chunk in enumerate(chunks, start=1):
    markdown_buffer.write(f"* {i}: [{chunk.web.title}]({chunk.web.uri})\n")


Markdown(markdown_buffer.getvalue())

In [ ]:
# Dynamic grounding¶
# In a context where you may not know in advance whether to enable search grounding or not, you can provide the model with a threshold 
# over which it will use search grounding.
# This is helpful in conversational contexts, where not every turn of conversation requires search data to support a response.

nosearch_model = genai.GenerativeModel("gemini-1.5-flash-002")
chat = nosearch_model.start_chat()

# No search grounding.
r = chat.send_message("Hello friendly chatbot!")

# Enable search for just this turn.
r = chat.send_message(
    "Who took home the 2023 cricket world cup?", tools="google_search_retrieval"
)

Markdown(r.text)

In [ ]:
HTML(r.candidates[0].grounding_metadata.search_entry_point.rendered_content)

In [ ]:
# The dynamic retrieval score is a probability, so the threshold
# must also be bound by [0, 1].
search_config = {
    "dynamic_retrieval_config": {"mode": "MODE_DYNAMIC", "dynamic_threshold": 0.5}
}

maybe_search_model = genai.GenerativeModel(
    "gemini-1.5-flash-002", tools={"google_search_retrieval": search_config}
)

chat = maybe_search_model.start_chat()

r = chat.send_message("Hello friendly chatbot!")
rc = r.candidates[0]
score = rc.grounding_metadata.retrieval_metadata.google_search_dynamic_retrieval_score
print(f"First turn: {score=}")

r = chat.send_message("Who took home the 2023 cricket world cup?")
rc = r.candidates[0]
score = rc.grounding_metadata.retrieval_metadata.google_search_dynamic_retrieval_score
print(f"Second turn: {score=}")
print()

display(Markdown(r.text))

In [ ]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)